## Creating a Web Data Project with Jupyter Notebooks
******
###### 1. Read in json file(s) resulting from your work with requests/selenium and beautifulsoup

In [11]:
# Import necessary packages
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

# Read in json files and create dataframes
filename5 = '2017-03-20.PluginAllianceSales.json'
data5 = pd.read_json(filename5)
filename6 = '2017-03-21.PluginAllianceSales.json'
data6 = pd.read_json(filename6)

#filename = '2017-01-24.PluginSales.json'
#data = pd.read_json(filename)
#filename2 = '2017-01-31.PluginSales.json'
#data2 = pd.read_json(filename2)
#filename3 = '2017-02-02.PluginSales.json'
#data3 = pd.read_json(filename3)
#filename4 = '2017-02-21.PluginAllianceSales.json'
#date4 = pd.read_json(filename4)

data5


,Brand,Current Price,Original Price,Product Name,Teaser
0,Plugin Alliance,"$3,659","$8,663",100% AAX DSP Bundle V1.5,50 world-class AAX DSP plugins. 10 highly resp...
1,Plugin Alliance,"$4,739","$11,766",All Bundle V4.6,75 world-class plugins from 15 highly respecte...
2,SPL,$799,"$1,750",100% SPL Bundle V2.2,"Transient Designer Plus, Vitalizer, De-Essers,..."
3,Brainworx,"$2,109","$4,477",100% BX Bundle V2.9,Get all of the award-winning Brainworx plugins...
4,Brainworx,$229,$317,BX Mega Amp Bundle V1.0,Faithful emulations of perhaps the most popula...
5,elysia,$619,$946,100% elysia Bundle V1.1,From the flagship alpha compressor to the powe...
6,ENGL,$229,$317,E765/E646 Amp Bundle - V1.0,The ENGL Amp Bundle brings to the native plugi...
7,Lindell Audio,$419,$634,100% Lindell Bundle V1.1,Includes the brand new TE-100 EQ and four more...
8,Lindell Audio,$249,$336,ChannelX Bundle,"Includes the 7X-500 limiter, the PEX-500 EQ an..."
9,Plugin Alliance,"$3,659","$8,663",100% AAX DSP Bundle V1.5,50 world-class AAX DSP plugins. 10 highly resp...


###### 2. Clean the data column by column
- Ensure that addresses (or other text data) are consistent 
- Eliminate string characters from numeric values 
- Exclude redundant data as appropriate

In [12]:
# Eliminate string characters from numeric data
data5['Original Price'] = data5['Original Price'].str.replace('$','')
data5['Current Price'] = data5['Current Price'].str.replace('$','')
data5['Original Price'] = data5['Original Price'].str.replace(',','')
data5['Current Price'] = data5['Current Price'].str.replace(',','')

data6['Original Price'] = data6['Original Price'].str.replace('$','')
data6['Current Price'] = data6['Current Price'].str.replace('$','')
data6['Original Price'] = data6['Original Price'].str.replace(',','')
data6['Current Price'] = data6['Current Price'].str.replace(',','')


In [13]:
#set(data5['Original Price'])
#set(data5['Current Price'])

In [14]:
#set(data6['Original Price'])
#set(data6['Current Price'])

In [15]:
# Exclude redundant rows
NoDupsData = data5.drop_duplicates()
NoDupsData2 = data6.drop_duplicates()

###### 3. Merge dataframes from separate json files as appropriate
- Find the intersection of two (or more) sets
- Compare the intersection with the newer set to find 'New Products'  
    - When found, add the starting date
- Compare the intersection with the older set to find 'Closed Products'  
    - When found, add the closing date
    - Compare closing date with starting date to find days on market      

In [18]:
#Create a dataframe containing the intersection of the two.
dataInnerMerge = pd.merge(NoDupsData, NoDupsData2, \
                          on=['Brand','Current Price','Original Price','Product Name', 'Teaser'],\
                          how='inner')

In [19]:
# Determine the set of 'new products'
NoDupsData['key1'] = 1
dataInnerMerge['key2'] = 1
newProducts = pd.merge(NoDupsData, dataInnerMerge, \
                      on=['Brand','Current Price','Original Price','Product Name', 'Teaser'], \
                      how = 'left')

# Create the associated dataframe
newProducts = newProducts[newProducts.key2 != newProducts.key1]

# Add the starting date in the dataframe in which the product is first shown

/Users/apple/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


AttributeError: 'DataFrame' object has no attribute 'key1'

###### 4. Create visualizations
- Histogram of Prices
- Histogram of Days on Market
- Scatter Diagram of Prices vs Days on Market
- Pie Chart of New, Like New, Used

###### 5. Trends from your data  
- Search for specific brands and offer counts for each
- Search for product types and offer counts for each
- For each of the above (and other) give counts for day over day or week over week

In [20]:
# List the products representing specific brands of your choosing
# for each date provided
brainWorxData = NoDupsData[NoDupsData['Brand'].str.contains('Brainworx ')]
unfilteredAudioData = NoDupsData[NoDupsData['Brand'].str.contains('Unfiltered Audio')]
pluginAllianceData = NoDupsData[NoDupsData['Brand'].str.contains('Plugin Alliance')]
brainWorxData2 = NoDupsData2[NoDupsData['Brand'].str.contains('Brainworx ')]
unfilteredAudioData2 = NoDupsData2[NoDupsData['Brand'].str.contains('Unfiltered Audio')]
pluginAllianceData2 = NoDupsData2[NoDupsData['Brand'].str.contains('Plugin Alliance')]

# Create a bar chart of products by brand (for the brands chosen)
brainWorxCount = len(brainWorxData)
unfilteredAudioCount = len(unfilteredAudioData)
pluginAllianceCount = len(pluginAllianceData)
brainWorxCount2 = len(brainWorxData2)
unfilteredAudioCount2 = len(unfilteredAudioData2)
pluginAllianceCount2 = len(pluginAllianceData2)

objects = ('BrainWorx','BrainWorx2','', 'UnfilteredAudio', 'UnfilteredAudio2','', \
           'PluginAlliance', 'PluginAlliance2')
colors = ('green','red','blue','green','red','blue','green','red')
y_pos = np.arange(len(objects))
performance = [brainWorxCount,brainWorxCount2,0,unfilteredAudioCount,unfilteredAudioCount2,\
               0,pluginAllianceCount,pluginAllianceCount2]
fig = plt.figure(figsize=(15,7))
plt.bar(y_pos, performance, align='center',alpha=0.5, color=colors)
plt.xticks(y_pos, objects)
plt.ylabel('Counts')
plt.title('Brands')
 
plt.show()


/Users/apple/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


IndexingError: Unalignable boolean Series key provided